In [4]:
# %pip install -q google-cloud-aiplatform==1.36.0 langchain==0.0.327 unstructured chromadb==0.4.15 --upgrade --user "unstructured[pdf]" pydantic gradio pdf2image google-cloud-vision google-cloud-storage pymupdf easyocr numpy python-docx python-pptx

Note: you may need to restart the kernel to use updated packages.


In [6]:
# !pip install --upgrade google-cloud-aiplatform

# Imports

In [1]:
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI

# # Chroma DB as Vector Store Database
from langchain.vectorstores import Chroma
import chromadb

from langchain.chains import RetrievalQA
import gradio as gr

# Global vars & Models

In [2]:
DB_PATH = "chroma_db5"
CHROMA_DB_CLIENT = chromadb.PersistentClient(path = DB_PATH)
QA= None
CACHE_COLLECTION_NAME = None

EMBEDDING = VertexAIEmbeddings()
# EMBEDDING = VertexAIEmbeddings(model_name='text-embedding-004')
LLM = VertexAI(
    model_name="text-bison-32k",
    # model_name = "gemini-1.0-pro",
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.6,
    top_k=40,
    verbose=True,
)

2024-09-25 19:28:12.566513: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-25 19:28:13.852622: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2024-09-25 19:28:13.852778: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

# Retrive Answer

In [3]:
def get_retrieval_qa(collection_name):
    db = Chroma(persist_directory=DB_PATH, collection_name=collection_name, embedding_function=EMBEDDING)
    # retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})
    retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 4})
    qa = RetrievalQA.from_chain_type(llm=LLM, chain_type="stuff", retriever=retriever, return_source_documents=True)
    return qa

def process_question(question, collection_name):
    global QA, CACHE_COLLECTION_NAME
    try:
        if (not CACHE_COLLECTION_NAME) or (collection_name!=CACHE_COLLECTION_NAME):
            QA = get_retrieval_qa(collection_name)
            CACHE_COLLECTION_NAME = collection_name
        response = QA({"query": question})
        return response["result"], response["source_documents"][0]
    except Exception as e:
        print(f"Error processing question: {e}")
        return "Error processing question", []

# Vector DB Operations

In [4]:
def get_all_collections():
    # Function to get the list of all collections
    collections = CHROMA_DB_CLIENT.list_collections()
    collection_names = [col.name for col in collections]
    return collection_names

def get_files_in_collection(collection_name):
    # Function to get the list of files in a given collection
    try:
        collection = CHROMA_DB_CLIENT.get_collection(collection_name)
        documents = collection.get()
        
        # Prepare a list to store file details
        file_details = {}
        
        for i in range(len(documents['ids'])):
            file_details[documents['metadatas'][i]['file_name']] = {'Upload Date Time': documents['metadatas'][i]['upload_date_time']}
        
        return file_details
    
    except Exception as e:
        return str(e)

# UI: Gradio Interface

In [ ]:
with gr.Blocks() as app:
    gr.Markdown("# OCR_based_gpt")
    gr.Markdown("Interact with your chatbot here! Ask questions based on the content of the files.")

    with gr.Row():
        with gr.Column():
            # Q&A Section
            gr.Markdown("## Ask Your Question")
            
            # Inputs for the question and collection name
            question_input = gr.Textbox(label="Enter your question")
            collection_input = gr.Textbox(label="Collection Name", placeholder="Enter Collection Name")
            
            # Button to submit the question
            submit_btn = gr.Button("Submit Question")
            
            # Outputs for the response and source document
            response_output = gr.Textbox(label="Chatbot Response")
            source_doc_output = gr.JSON(label="Source Document")
            
            submit_btn.click(
                process_question,
                inputs=[question_input, collection_input],
                outputs=[response_output, source_doc_output]
            )
    
        with gr.Column():
            # List Collections Section
            gr.Markdown("## Get All Collections")
            all_collections_output = gr.JSON(label="All Collections")
            gr.Button("List All Collections").click(
                get_all_collections,
                inputs=[],
                outputs=all_collections_output
            )
        
            # List Files in Collection Section
            gr.Markdown("## Get Files in Collection")
            collection_name_input = gr.Textbox(label="Collection Name", placeholder="Enter Collection Name")
            files_output = gr.JSON(label="Files in Collection")
            gr.Button("List Files in Collection").click(
                get_files_in_collection,
                inputs=collection_name_input,
                outputs=files_output
            )

# Launch the Gradio app
app.launch(share=True, debug=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://1224ea2fe39f5216f9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Architecture Diagram of OCR based chatbot? 

What ChromaDB client? 

What is text_splitter? 

What is upload and process? 

What is delete collection?